In [1]:
from transformers import AutoTokenizer, AutoModel

# Load pre-trained model and tokenizer
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Specify the directory path where you want to save the model and tokenizer
save_directory = "/Users/david/Desktop/QA Media/Models"

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

('/Users/david/Desktop/QA Media/Models/tokenizer_config.json',
 '/Users/david/Desktop/QA Media/Models/special_tokens_map.json',
 '/Users/david/Desktop/QA Media/Models/vocab.txt',
 '/Users/david/Desktop/QA Media/Models/added_tokens.json',
 '/Users/david/Desktop/QA Media/Models/tokenizer.json')

In [2]:
#Import Modules
import requests
from bs4 import BeautifulSoup
import pandas as pd

articles = {}

# Load the Excel file
df = pd.read_excel('article urls.xlsx')

main_url = "https://www.qa-financial.com"

# Convert the DataFrame to a list
urls = df['URL'].tolist()  # Replace 'urls' with the actual column name in the Excel file

# For each URL, get the text content
for url in urls:
    try:
        response = requests.get(main_url + url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get the title and text based on your description
        #title = soup.find('title').get_text()
        text = soup.find('meta', attrs={'name': 'description'}).get('content')

        articles[main_url + url] = text
    except:
        continue
        
import json

# Save the dictionary to a file
with open('dict_file.json', 'w') as file:
    json.dump(articles, file)

In [3]:
article_content = [x for x in articles.values()]

# Function to get embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

# Assume `documents` is a list of your news articles
embeddings = [get_embeddings(doc) for doc in article_content]

In [4]:
import pickle

# Save the list of arrays.
with open('arrays.pkl', 'wb') as f:
    pickle.dump(embeddings, f)
